<a href="https://colab.research.google.com/github/DrakeShadowRaven/desihigh/blob/main/n'ish_body.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')
import math

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/desihigh')

In [3]:
import time
import astropy
import itertools
import matplotlib

import numpy             as np
import pylab             as pl
import matplotlib.pyplot as plt
import astropy.units     as u

from   astropy.cosmology import FlatLambdaCDM
from   IPython.display   import YouTubeVideo
from   tools.flops       import flops

In [4]:
from IPython.display import clear_output
from time import sleep

In [5]:
%matplotlib inline

plt.style.use('dark_background')

In [6]:
def init_dof(npt=1):
  #  Create a set of particles at random positions in a box, which will soon predict the distribution of dark matter 
  #  as we see above.
  xs     = np.random.uniform(0., 1., npt)
  ys     = np.random.uniform(0., 1., npt)
  zs     = np.random.uniform(0., 1., npt)

  pos    = np.vstack((xs,ys,zs)).T
  vel    = np.zeros_like(pos)

  return  pos, vel

In [29]:
# Reinitialise particles.
npt=2
pos, vel = init_dof(npt=npt)
mass = np.random.uniform(0., 1., npt)
msize = mass*20
print, mass

# force the last mass very small


print,mass
# pos of partile 1


r_2 = .3e12
r = 1.2e12
mass_1 = 1.5e30
mass_2 = 6e30
v_1x = 0
v_1y = 13200
v_2x = 0
v_2y= -3300
print(r)
print(mass_1)
print(mass_2)


G = 6.67e-11


p_1x = mass_1*v_1x
p_2x = mass_2*v_2x
p_1y = mass_1*v_1y
p_2y = mass_2*v_2y
print(p_1x)
print(p_2x)
print(p_1y)
print(p_2y)


modv1 = math.sqrt(v_1x**2+v_1y**2)
modv2 = math.sqrt(v_2x**2+v_2y**2)

E = .5*mass_1*v_1y**2+.5*mass_2*v_2y**2-(G*mass_1*mass_2)/(r)
print(E)
L = mass_1*v_1y*r-mass_2*v_2y*r_2
print(L)
M= 7.5e30

mu = (mass_1*mass_2)/(mass_1+mass_2)
print(mu)
a=2*(L**2)*E
b=(mu**3)*(G**2)*M**2
A=a/b
print(A)




1200000000000.0
1.5e+30
6e+30
0.0
0.0
1.9799999999999997e+34
-1.9799999999999997e+34
-3.3689999999999997e+38
2.9699999999999997e+46
1.1999999999999998e+30
-1.3744405907990538


In [22]:
def init_dof(npt=1):
  #  Create a set of particles at random positions in a box, which will soon predict the distribution of dark matter 
  #  as we see above.
  xs     = np.random.uniform(0., 1., npt)
  ys     = np.random.uniform(0., 1., npt)
  zs     = np.random.uniform(0., 1., npt)

  pos    = np.vstack((xs,ys,zs)).T
  vel    = np.zeros_like(pos)

  return  pos, vel

In [27]:

# Number and mass of particles
npt        = 3
#mass       = 0.25

# Whether to draw arrows for the acceleration and velocity
draw_acc   = False
#draw_vel   = False
draw_vel = True

# A small drag term to simulate the real drag dark matter particles experience due to the expanding universe
drag = 1e-2
dt         = 5e-4
nsteps     = 500
timesteps  = np.linspace(0, (nsteps)*dt, nsteps, endpoint=False)
def make_legend_arrow(legend, orig_handle,
          xdescent, ydescent,
          width, height, fontsize):
    p = matplotlib.patches.FancyArrow(0, 0.5*height, width, 0, length_includes_head=True, head_width=0.75*height)
    return p

for index_in_timestep, time in enumerate(timesteps):
    clear_output(wait=True)
    if index_in_timestep%10 == 1:
        fig, ax = plt.subplots(1,1, figsize=(5,5), dpi=150)
        ax.tick_params(labelbottom=False, labelleft=False, left=False, bottom=False)
        ax.clear()
        ax.set_title(f'N-body simulation with $N={npt}$ particles')
        step_label = ax.text(0.03, .97, f'Step {index_in_timestep}',
            transform=ax.transAxes, verticalalignment='top', c='k',
            bbox=dict(color='w', alpha=0.8))
        
    dvel                        = np.zeros_like(vel)
    dpos                        = np.zeros_like(pos)
    acc                         = np.zeros_like(pos)
    for index_in_particle in range(npt):
        acc[index_in_particle]  = g_at_pos(pos[index_in_particle], pos, mass, epsilon=0.1)
        # Update velocities.
        dvel[index_in_particle] = dt * acc[index_in_particle]
        # Update positions.
        dpos[index_in_particle] = dt * vel[index_in_particle]
    vel += dvel - drag*vel
    pos += dpos
    # Our simulation has periodic boundaries, if you go off one side you come back on the other!
    pos = pos % 1
    ax.scatter(pos[:,0], pos[:,1], s=msize, color='darkorange', edgecolor='w')
    # Draw arrows representing the velocity and acceleration vectors, if requested
    # The code here is a little verbose to get nice-looking arrows in the legend
    arrows = []
    if draw_vel:
        ax.quiver(pos[:,0], pos[:,1], vel[:,0], vel[:,1], color='w', zorder=0)
        arrows += [matplotlib.patches.FancyArrow(0,0, 0.5, 0.6, label='Velocity', color='w')]
    if draw_acc:
        ax.quiver(pos[:,0], pos[:,1], acc[:,0], acc[:,1], color='darkorange', zorder=0)
        arrows += [matplotlib.patches.FancyArrow(0,0, 0.5, 0.6, label='Accel', color='darkorange')]
    if draw_vel or draw_acc:
        ax.legend(handles=arrows, handler_map={matplotlib.patches.FancyArrow:matplotlib.legend_handler.HandlerPatch(patch_func=make_legend_arrow)},
                 facecolor='k', edgecolor='white', framealpha=0.8,
                 loc='lower right')
#    fig.canvas.draw()
    if index_in_timestep%10 == 1:
      ax.set_xlim(0.5, 0.7)
      ax.set_ylim(0.4, 0.6)
      plt.show(fig)
      sleep(0.05)

NameError: ignored